In [1]:
# import warnings
# warnings.simplefilter('ignore')
# warnings.filterwarnings('ignore')
# warnings.filterwarnings(action='once')


import spacy
# gpu = spacy.prefer_gpu()
# print('GPU:', gpu)
# pip install -U spacy[cuda100]
# python -m spacy validate
# python -m spacy download en_core_web_sm

from spacy import displacy
from collections import Counter
import en_core_web_sm
from pprint import pprint

nlp = en_core_web_sm.load()
import re
from stopwords import *
import nltk
from preprocess import *
# from feature import *

from textblob import TextBlob
import collections

from spacy.symbols import cop, acomp, amod, conj, neg, nn, nsubj, dobj,prep,advmod
from spacy.symbols import VERB, NOUN, PROPN, ADJ, ADV, AUX, PART

pattern_counter = collections.Counter()

from nltk.corpus import stopwords
import networkx as nx
from MongoDB import MongoDB
import os
import pickle

D:\Program\Anaconda3\envs\tensorflow\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\Program\Anaconda3\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
D:\Program\Anaconda3\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


# 斷詞辭典

In [2]:
from nltk.corpus import stopwords
stops = set(stopwords.words("english"))
stpwords_list3 = [f.replace("\n","") for f in open("stopwords.txt","r",encoding = "utf-8").readlines()]
stpwords_list3.remove("not")
stopwords = list(html_escape_table + stpwords_list2) + list(list(stops) + list(stpwords_list1) + list(stpwords_list3))

# Total Opinion
# https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html#datasets
<!-- Opinion Lexicon: A list of English positive and negative opinion words or sentiment words (around 6800 words). This list was compiled over many years starting from our first paper (Hu and Liu, KDD-2004). -->

In [3]:
opinion_lexicon = {}
for filename in os.listdir('opinion-lexicon-English/'):      
    if "txt" not in filename: continue
    print(filename)
    with open('opinion-lexicon-English/'+filename,'r') as f_input:
        lexion = []
        for line in f_input:
            if line.startswith(";"):
                continue
            word = line.replace("\n","")
            if word != "" : lexion.append(word)
        pos = filename.replace(".txt","")
        opinion_lexicon[pos] = lexion

opinion_lexicon["total-words"] = opinion_lexicon["negative-words"] + opinion_lexicon["positive-words"]
# opinion_lexicon["total-words"]

negative-words.txt
positive-words.txt


In [4]:
# from nltk.lm import Vocabulary
# vocab = Vocabulary(words, unk_cutoff=2)


# pickle 提取

In [5]:
# reload a file to a variable    
from glob import glob
with open(glob('data/new_*_*.pickle')[1], 'rb') as file:
    print(file.name)
    PROD_DICT = pickle.load(file)
    category1,category2 = file.name.replace(".pickle","").split("_")[1:]
    
category1,category2

data\new_Electronics_DVD Players.pickle


('Electronics', 'DVD Players')

# 句子篩選/切割

In [6]:
import textacy
from summa import keywords,summarizer

def sentence_extract_blob(text):    
    extract_sents = []
    text = text.replace(",","").replace("i.e.","")
    text = SentProcess(text)
    
    for sent in text.split("<end>"):        
#         print(sent)
        if sent == "": continue
#         sentiment = TextBlob(sent).sentences[0].sentiment
#         if abs(sentiment.polarity) <= 0.5: continue
#         if abs(sentiment.subjectivity) <= 0.5: continue
#         print(TextBlob(sent).sentences)
#         print("polarity : ",sentiment.polarity)
#         print("subjectivity : ",sentiment.subjectivity)
# #         print(keywords.keywords(sent))
#         print("***")
        extract_sents.append(sent)
    return extract_sents

def extract_cand_pharse(text): 
    sent_pattern = r"""( 
    (<NOUN><NOUN> | <ADJ><NOUN> | <NOUN><NOUN><NOUN> | <NOUN><ADP><DET><NOUN> | 
    <NOUN> | <ADJ><NOUN><NOUN> | <ADJ><ADJ><NOUN> | <NOUN><PREP><NOUN> | <NOUN><PREP><DET><NOUN> | 
    <VERB><PUNCT><ADP><NOUN><NOUN> | <DET><NOUN> )


    (<AUX><ADV><ADJ>|<AUX><ADJ>|<ADV><ADJ>|<ADJ><NOUN>) 
    )
    """
#     pattern = r'<PROPN>+ (<PUNCT|CCONJ> <PUNCT|CCONJ>? <PROPN>+)*'
    extract_pharse = []
    doc = textacy.make_spacy_doc(text,lang='en_core_web_sm')
    phrases = textacy.extract.pos_regex_matches(doc, sent_pattern)
    # Print all Verb Phrase

    for phrase in phrases:
#         print("verb_phrases : " , phrase.text)
    #     print([(token.text,token.pos_) for token in nlp(chunk.text)])
        extract_pharse.append(phrase.text+"\n")           
    return extract_pharse        


# 規格關鍵字提取

In [7]:
def getKeywordFeat(description,feature):  # 搜索關鍵保留字，並重新清理句子       
    description_list = []
    for sent in description:
        if len(TextBlob(sent)) > 1:
            sent_list = sent.split(".")
            description_list.extend(sent_list) 
        else:
            description_list.extend(sent) 
    description_list = [sent.strip() +"." for sent in description_list]
    if type(feature) == str:
        total_desc_sents = description_list + [feature]
    else:
        total_desc_sents = description_list + feature    
 
    # Stage 1 取dash feature
    keywords = []
    newdescription = ""
    
    for sent in total_desc_sents:
        keys = []
        for token in nlp(sent):            
            sent = sent.lower()
            for k, v in contractions.items():
                if k in sent:
                    sent = sent.replace(k, v)

            for k in html_escape_table:
                if k in sent:
                    sent = sent.replace(k, "")            
            
        pattern = re.compile(r"([\d\w\.-]+[-'//.][\d\w\.-]+)")  # |  ([\(](\w)+[\)])
        keys = pattern.findall(sent)            
            
        keywords.extend(keys)
        cleansent = sent
        for k in keys:    
            k = k.replace("(","").replace(")","")
            cleansent = cleansent.replace(k,"(" + k + ")")
            cleansent = cleansent.replace("((","(").replace("))",")")     
            cleansent = cleansent.replace("(" + k + ")","") # 移除符號特徵
            cleansent = cleansent.strip()
        
#         cleansent = cleansent + ".\n"
        cleansent = remove_word3(cleansent)
        char = " "
        while char * 2 in cleansent:
            cleansent = cleansent.replace(char * 2, char)  
        char = "."
        while char * 2 in cleansent:
            cleansent = cleansent.replace(char * 2, char)            
         
        if len(cleansent) < 5 : continue
        newdescription = newdescription + cleansent + ".\n"
        
    newdescription = newdescription.replace("..",".")   
    newdescription = re.sub(r"\([\w]+\)","",newdescription)
    newdescription = re.sub(r"\(\)","",newdescription)   
    newdescription = re.sub(r'\"',"",newdescription)
    
    # get Noun pharse keyword for newdescription
#     for pharse in nlp(newdescription).noun_chunks:
#         print(pharse.text)
    cand_pf = PF_rule_POS(newdescription).run()
#     print(cand_pf)
    
    keywords2 = set()
    for pf in cand_pf:
        chunk_pfs = nltk.word_tokenize(pf)
        for tok in chunk_pfs:
            if tok in stopwords: continue
            if tok in opinion_lexicon["total-words"]: continue
            keywords2.add(tok)

#     print(len(keywords2),keywords2)
    newkeywords = list()
    for key in list(keywords) + list(keywords2):
        clean_key = remove_word2(key)
        clean_key = clean_key.split(" ")            
        newkeywords.extend(clean_key) 
    newkeywords = list(set([key for key in newkeywords if len(key) >=2 and key not in stopwords and key not in opinion_lexicon["total-words"]]))
    
    return keywords,keywords2,newkeywords,newdescription

In [8]:
def getKeywordFeat_2(article):  # 搜索關鍵保留字，並重新清理句子       
   
    total_desc_sents = sentence_extract_blob(article)
    # Stage 1 取dash feature
    keywords = []
    newarticle = ""
    
    for sent in total_desc_sents:
        keys = []
        for token in nlp(sent):            
            sent = sent.lower()
            for k, v in contractions.items():
                if k in sent:
                    sent = sent.replace(k, v)

            for k in html_escape_table:
                if k in sent:
                    sent = sent.replace(k, "")            
            
        pattern = re.compile(r"([\d\w\.-]+[-'//.][\d\w\.-]+)")  # |  ([\(](\w)+[\)])
        keys = pattern.findall(sent)            
            
        keywords.extend(keys)
        cleansent = sent
        for k in keys:    
            k = k.replace("(","").replace(")","")
            cleansent = cleansent.replace(k,"(" + k + ")")
            cleansent = cleansent.replace("((","(").replace("))",")")     
            cleansent = cleansent.replace("(" + k + ")","") # 移除符號特徵
            cleansent = cleansent.strip()
        
#         cleansent = cleansent + ".\n"
        cleansent = remove_word3(cleansent)
        char = " "
        while char * 2 in cleansent:
            cleansent = cleansent.replace(char * 2, char)  
        char = "."
        while char * 2 in cleansent:
            cleansent = cleansent.replace(char * 2, char)            
         
        if len(cleansent) < 5 : continue
        newarticle = newarticle + cleansent + ".\n"
        
    newarticle = newarticle.replace("..",".")   
    newarticle = re.sub(r"\([\w]+\)","",newarticle)
    newarticle = re.sub(r"\(\)","",newarticle)   
    newarticle = re.sub(r'\"',"",newarticle)
    
    # get Noun pharse keyword for newarticle
#     for pharse in nlp(newarticle).noun_chunks:
#         print(pharse.text)
    cand_pf = PF_rule_POS(newarticle).run()
#     print(cand_pf)
    
    keywords2 = set()
    for pf in cand_pf:
        chunk_pfs = nltk.word_tokenize(pf)
        for tok in chunk_pfs:
            if tok in stopwords: continue
            if tok in opinion_lexicon["total-words"]: continue
            keywords2.add(tok)
            
    newkeywords = list()
    for key in list(keywords) + list(keywords2):
        clean_key = remove_word2(key)
        clean_key = clean_key.split(" ")            
        newkeywords.extend(clean_key) 
    newkeywords = list(set([key for key in newkeywords if len(key) >=2 and key not in stopwords and key not in opinion_lexicon["total-words"]]))
    

#     print(len(keywords2),keywords2)
    return keywords,keywords2,newkeywords,newarticle

# 評論濾除符號-英文特徵

In [9]:
def cleanReview(review):  # 搜索關鍵保留字，並重新清理句子     
    # Stage 1 取dash feature
    keywords = []
    newReview = ""
    for sent in nltk.sent_tokenize(review):
        for token in nlp(sent):
            sent = sent.lower()
            for k, v in contractions.items():
                if k in sent:
                    sent = sent.replace(k, v)

            for k in html_escape_table:
                if k in sent:
                    sent = sent.replace(k, "")
            
            
            pattern = re.compile(r"([\d\w\.-]+[-'//.][\d\w\.-]+)")  # |  ([\(](\w)+[\)])
            keys = pattern.findall(sent)
            
            
        keywords.extend(keys)
        cleansent = sent
        for k in keys:    
            k = k.replace("(","").replace(")","")
            cleansent = cleansent.replace(k,"(" + k + ")")
            cleansent = cleansent.replace("((","(").replace("))",")")     
            cleansent = cleansent.replace("(" + k + ")","") # 移除符號特徵
            cleansent = cleansent.strip()
        
#         cleansent = cleansent + ".\n"
        cleansent = remove_word3(cleansent)
        char = " "
        while char * 2 in cleansent:
            cleansent = cleansent.replace(char * 2, char)  
        char = "."
        while char * 2 in cleansent:
            cleansent = cleansent.replace(char * 2, char)            
         
        if len(cleansent) < 5 : continue
        newReview = newReview + cleansent + ".\n"
        
    newReview = newReview.replace("..",".")   
    newReview = re.sub(r"\([\w]+\)","",newReview)
    newReview = re.sub(r"\(\)","",newReview)   
    newReview = re.sub(r'\"',"",newReview)
    
    # get Noun pharse keyword for newdescription
#     cand_pf = PF_rule_POS(newReview).run()
    
#     keywords2 = set()
#     for pf in cand_pf:
#         chunk_pfs = nltk.word_tokenize(pf)
#         for tok in chunk_pfs:
#             if tok in stopwords: continue
#             if tok in opinion_lexicon["total-words"]: continue
#             keywords2.add(tok)

#     print(len(keywords2),keywords2)
    return keywords, newReview

In [10]:
review = """
I was a happy S530D owner until a month ago when I began getting the dreaded \"C13 00\" error.  It would not recognize any DVD I put in; even brand new ones.\n\nI did a web search and was amazed to find that many, many people who have bought this player (and Sony players in general) have had exactly the same problem after a year to a year and half.\n\nSony refuses to do a recall and I've since bought a new player (not a Sony; I settled on a JVC XV-FA90), but I thought I should post my experience here.  Buyer beware on this player; it has a very high rate of terminal failure.
    
"""


rev_dash_keywords, newreview = cleanReview(review)
print(newreview)

i was happy sd owner until month ago when began getting the dreaded error.
it would not recognize any dvd put in even brand new ones.
did web search and was amazed to find that many many people who have bought this player and sony players in general have had exactly the same problem after year to year and half.
sony refuses to do recall and have since bought new player not sony settled on jvc but thought should post my experience here.
buyer beware on this player it has very high rate of terminal failure.



# PF-Extraction-Rule(POS)

In [11]:
# -NN, 
# -NN NN, 
# JJ NN 
# -NN NN NN, 
# JJ NN NN, 
# JJ JJ NN, 
# NN IN NN 
# -NN IN DT NN

class PF_rule_POS():
    def __init__(self, article):
        self.article = article
        self.matched_sents = []  # Collect data of matched sentences to be visualized

    def collect_sents(self, matcher, doc, i, matches):
        match_id, start, end = matches[i]
        span = doc[start:end]  # Matched span
        sent = span.sent  # Sentence containing matched span
        # Append mock entity for match in displaCy style to matched_sents
        # get the match span by ofsetting the start and end of the span with the
        # start and end of the sentence in the doc
        match_ents = [{
            "start": span.start_char - sent.start_char,
            "end": span.end_char - sent.start_char,
            "label": "MATCH",
        }]
        self.matched_sents.append({"text": sent.text, "ents": match_ents})

    def match_pattern(self, sent, flit_keyword=None):
        res = []
        #         ('Dolby', 'PROPN'), ('Digital', 'PROPN')
        matcher = Matcher(nlp.vocab)
        #         matcher.add("pf1", self.collect_sents, [{'POS': {"IN": ['NOUN', 'PROPN']}}, {'IS_PUNCT': True},
        #                                                    {'POS': {"IN": ['NOUN', 'PROPN']}}])  # add pattern
        #         matcher.add("pf2", self.collect_sents,
        #                     [{'POS': {"IN": ['NOUN', 'PROPN']}}, {'IS_PUNCT': True}, {'POS': {"IN": ['NOUN', 'PROPN']}},
        #                      {'POS': 'SYM'}, {'POS': {"IN": ['NOUN', 'PROPN']}}])  # add pattern
        #         #         matcher.add("specn3", self.collect_sents, [{'POS': 'ADJ'}, {'POS': 'NOUN'}])  # add pattern
        #         matcher.add("pf3", self.collect_sents,
        #                     [{'POS': {"IN": ['NOUN', 'PROPN']}}, {'POS': {"IN": ['NOUN', 'PROPN']}}])  # add pattern
        #         matcher.add("pf4", self.collect_sents,
        #                     [{'POS': {"IN": ['NOUN', 'PROPN']}}, {'POS': {"IN": ['NOUN', 'PROPN']}},
        #                      {'POS': {"IN": ['NOUN', 'PROPN']}}])  # add pattern
        #         matcher.add("pf5", self.collect_sents, [{'POS': 'ADJ'}, {'POS': {"IN": ['NOUN', 'PROPN']}},
        #                                                    {'POS': {"IN": ['NOUN', 'PROPN']}}])  # add pattern
        #         matcher.add("pf6", self.collect_sents,
        #                     [{'POS': 'ADJ'}, {'POS': 'ADJ'}, {'POS': {"IN": ['NOUN', 'PROPN']}}])  # add pattern
        #         #         ('inches', 'NOUN'), ('(', 'PUNCT'), ('W', 'NOUN'), ('x', 'SYM'), ('H', 'NOUN'), ('x', 'SYM'), ('D', 'NOUN'), (')', 'PUNCT')
        #         matcher.add("pf6.1", self.collect_sents,
        #                     [{'POS': {"IN": ['NOUN', 'PROPN']}}, {'IS_PUNCT': True}, {'POS': {"IN": ['NOUN', 'PROPN']}},
        #                      {'POS': 'SYM'}, {'POS': {"IN": ['NOUN', 'PROPN']}}, {'POS': 'SYM'},
        #                      {'POS': {"IN": ['NOUN', 'PROPN']}}, {'IS_PUNCT': True}])  # inches (W x H x D)
        #         matcher.add("pf6.2", self.collect_sents,
        #                     [{'POS': {"IN": ['NOUN', 'PROPN']}}, {'IS_PUNCT': True}, {'POS': {"IN": ['NOUN', 'PROPN']}},
        #                      {'POS': 'SYM'}, {'POS': {"IN": ['NOUN', 'PROPN']}}, {'IS_PUNCT': True}])  # inches (W x H)
        #         matcher.add("pf7", self.collect_sents, [{'POS': {"IN": ['NOUN', 'PROPN']}}, {'POS': 'PREP'},
        #                                                    {'POS': {"IN": ['NOUN', 'PROPN']}}])  # add pattern
        #         matcher.add("pf8", self.collect_sents, [{'POS': {"IN": ['NOUN', 'PROPN']}}, {'POS': 'PREP'}, {'POS': 'DET'},
        #                                                    {'POS': {"IN": ['NOUN', 'PROPN']}}])  # add pattern
        #         matcher.add("pf9", self.collect_sents,
        #                     [{'POS': 'VERB'}, {'POS': 'PUNCT'}, {'POS': 'ADP'}, {'POS': {"IN": ['NOUN', 'PROPN']}},
        #                      {'POS': {"IN": ['NOUN', 'PROPN']}}])  # add pattern

#         matcher.add("pf1", self.collect_sents, [{'TAG': {"IN": ['NN']}}])  # add pattern
        matcher.add("pf2", self.collect_sents, [{'TAG': {"IN": ['NN']}}, {'TAG': {"IN": ['NN']}}])  # add pattern
        matcher.add("pf3", self.collect_sents, [{'TAG': {"IN": ['JJ']}}, {'TAG': {"IN": ['NN']}}])  # add pattern    
        matcher.add("pf4", self.collect_sents,
                    [{'TAG': {"IN": ['NN']}}, {'TAG': {"IN": ['NN']}}, {'TAG': {"IN": ['NN']}}])  # add pattern
        matcher.add("pf5", self.collect_sents,
                    [{'TAG': {"IN": ['JJ']}}, {'TAG': {"IN": ['NN']}}, {'TAG': {"IN": ['NN']}}])
        matcher.add("pf6", self.collect_sents,
                    [{'TAG': {"IN": ['JJ']}}, {'TAG': {"IN": ['JJ']}}, {'TAG': {"IN": ['NN']}}])
        matcher.add("pf7", self.collect_sents,
                    [{'TAG': {"IN": ['NN']}}, {'TAG': {"IN": ['IN']}}, {'TAG': {"IN": ['NN']}}])
        matcher.add("pf8", self.collect_sents, [{'TAG': {"IN": ['NN']}}, {'TAG': {"IN": ['IN']}},
                                                {'TAG': {"IN": ['DT']}}, {'TAG': {"IN": ['NN']}}])

        doc = nlp(sent)
        matches = matcher(doc)
        # Serve visualization of sentences containing match with displaCy
        # set manual=True to make displaCy render straight from a dictionary
        # (if you're not running the code within a Jupyer environment, you can
        # use displacy.serve instead)
        #         displacy.render(self.matched_sents, style="ent", manual=True)
        for match_id, start, end in matches:
            # Get the string representation
            string_id = nlp.vocab.strings[match_id]
            span = doc[start:end]  # The matched span
            #             print(match_id, string_id, start, end, span.text)
            if flit_keyword:
                found = False
                spanStr = span.text
                for word in spanStr.split(" "):
                    if word in flit_keyword: found = True; break
                if found: res.append(span.text)
            else:
                res.append(span.text)
        return res

    def run(self):
        total_res = []
        for sent in nltk.sent_tokenize(self.article):
            sent = sent.lower()
            sent = sent.replace(",", " ").replace(":", " ").replace("(", " ").replace(")", " ").replace("multi-",
                                                                                                        "multiple ")
            ress = self.match_pattern(sent)
            total_res.extend(ress)

        total_res = list(set(total_res))
        #         print("final res",total_res)
        return total_res



# FO-Rule(POS)

In [12]:
class FO_rule_POS():
    def __init__(self, article):
        self.article = article
        self.matched_sents = []  # Collect data of matched sentences to be visualized

    def collect_sents(self, matcher, doc, i, matches):
        match_id, start, end = matches[i]
        span = doc[start:end]  # Matched span
        sent = span.sent  # Sentence containing matched span
        # Append mock entity for match in displaCy style to matched_sents
        # get the match span by ofsetting the start and end of the span with the
        # start and end of the sentence in the doc
        match_ents = [{
            "start": span.start_char - sent.start_char,
            "end": span.end_char - sent.start_char,
            "label": "MATCH",
        }]
        self.matched_sents.append({"text": sent.text, "ents": match_ents})

    def match_pattern(self, sent, flit_keyword=None):
        res = []
        #         ('Dolby', 'PROPN'), ('Digital', 'PROPN')
        matcher = Matcher(nlp.vocab)
        matcher.add("fos1", self.collect_sents, [{'TAG': {"IN": ['JJ']}},
                                                 {'TAG': {"IN": ['NN']}}])  # add pattern JJ[O] NN[F] 
        matcher.add("fos1.1", self.collect_sents, [{'TAG': {"IN": ['NN']}}, # picture/NN quality/NN great/JJ
                                                 {'TAG': {"IN": ['NN']}},
                                                  {'TAG': {"IN": ['JJ']}}])  # add pattern JJ[O] NN[F] 
               
        matcher.add("fos2", self.collect_sents,
                    [{'TAG': {"IN": ['VB']}},
                     {'TAG': {"IN": ['IN']}},
                     {'TAG': 'DT'},
                     {'TAG': {"IN": ['NN']}}])  # add pattern VB[O] NN[F]

        matcher.add("fos3", self.collect_sents,
                    [{'TAG': {"IN": ['JJ']}},
                     {'TAG': {"IN": ['CC']}},
                     {'TAG': {"IN": ['JJ']}},
                     {'TAG': {"IN": ['NN']}}])  # add pattern JJ[O] JJ[O] NN[F]

        matcher.add("fos4", self.collect_sents,
                    [{'TAG': {"IN": ['NN']}},
                     {'TAG': {"IN": ['VB']}},
                     {'TAG': {"IN": ['JJ']}}])  # add pattern NN[F] JJ[O]

        matcher.add("fos5", self.collect_sents,
                    [{'TAG': {"IN": ['NN']}},
                     {'TAG': {"IN": ['VB']}},
                     {'TAG': {"IN": ['RB']}}])  # add pattern NN[F] RB[O] 

        matcher.add("fos6", self.collect_sents,
                    [{'TAG': {"IN": ['NN']}},
                     {'TAG': {"IN": ['VB']}},
                     {'TAG': {"IN": ['RB']}},
                     {'TAG': {"IN": ['RB']}}])  # add pattern NN[F] RB[O]

        matcher.add("fos7", self.collect_sents,
                    [{'TAG': {"IN": ['NN']}},
                     {'TAG': {"IN": ['MD']}},
                     {'TAG': {"IN": ['RB']}},
                     {'TAG': {"IN": ['VB']}}])  # add pattern NN[F] VB[O]        

        matcher.add("fos8", self.collect_sents,
                    [{'TAG': {"IN": ['VB']}},
                     {'TAG': {"IN": ['DT']}},
                     {'TAG': {"IN": ['NN']}}])  # add pattern NN[F] VB[O]

        matcher.add("fos9", self.collect_sents,
                    [{'TAG': {"IN": ['NN']}},
                     {'TAG': {"IN": ['VBZ']}},
                     {'TAG': {"IN": ['JJR']}},
                     {'TAG': {"IN": ['IN']}},
                     {'TAG': {"IN": ['NN']}}])  # add pattern NN[F] JJR[O]

        matcher.add("fos10", self.collect_sents,
                    [{'TAG': {"IN": ['VB']}},
                     {'TAG': {"IN": ['NN']}}])  # add pattern NN[F] VB[O]    
        # -----------------------------------------------------------------------------
        matcher.add("p1", self.collect_sents, [{'TAG': {"IN": ['JJ']}},
                                               {'TAG': {"IN": ['NN',"NNS"]}}
                                               ])  
        matcher.add("p2", self.collect_sents, [{'TAG': {"IN": ['JJ']}},
                                               {'TAG': {"IN": ['NN',"NNS"]}},
                                               {'TAG': {"IN": ['NN',"NNS"]}}
                                               ])  
        matcher.add("p3", self.collect_sents, [{'TAG': {"IN": ['RB','RBR','RBS']}},
                                               {'TAG': {"IN": ['JJ']}},
                                               {'TAG': {"IN": ['NN',"NNS"]}}
                                               ]) 
#         matcher.add("p4", self.collect_sents, [{'TAG': {"IN": ['RB','RBR','RBS']}},
#                                                {'TAG': {"IN": ['RB','RBR','RBS']}},
#                                                {'TAG': {"IN": ['NN',"NNS"]}}
#                                                ])
        matcher.add("p5", self.collect_sents, [{'TAG': {"IN": ['RB','RBR','RBS']}},
                                               {'TAG': {"IN": ['RB','RBR','RBS']}},
                                               {'TAG': {"IN": ['JJ']}},
                                               {'TAG': {"IN": ['NN',"NNS"]}}
                                               ])
#         matcher.add("p6", self.collect_sents, [{'TAG': {"IN": ['RB','RBR','RBS']}},
#                                                {'TAG': {"IN": ['VBN','VBD','VB']}}                                               
#                                                ])
        matcher.add("p7", self.collect_sents, [{'TAG': {"IN": ['VBN','VBD','VB']}},
                                               {'TAG': {"IN": ['JJ']}}                                               
                                               ])    
#         matcher.add("p8", self.collect_sents, [{'TAG': {"IN": ['RB','RBR','RBS']}},
#                                                {'TAG': {"IN": ['RB','RBR','RBS']}},
#                                                {'TAG': {"IN": ['JJ']}}                                               
#                                                ])  
        matcher.add("p9", self.collect_sents, [{'TAG': {"IN": ['VBN','VBD','VB']}},
                                               {'TAG': {"IN": ['RB','RBR','RBS']}},
                                               {'TAG': {"IN": ['RB','RBR','RBS']}}
                                              ])

        doc = nlp(sent)
        matches = matcher(doc)

        for match_id, start, end in matches:
            # Get the string representation
            pattern_id = nlp.vocab.strings[match_id]
            span = doc[start:end]  # The matched span
            pos = [(token.text,token.tag_) for token in doc]
            span_pos = pos[start:end]
#             print(match_id, pattern_id, start, end, span.text)
#             print(span_pos)
            fos = self.match_fos(pattern_id,span_pos) 
#             print(fos)
            for fo in fos:
                if fo not in res: 
                    res.append(fo)
        return res
    
    def match_fos(self,pattern_id,span_pos):
        if pattern_id == "fos1": # add pattern JJ[O] NN[F]
            o = span_pos[0][0]
            f = span_pos[1][0]
        if pattern_id == "fos1.1": # add pattern JJ[O] NN[F]
            o = span_pos[2][0]
            f = "%s %s"%(span_pos[0][0] ,span_pos[1][0])
        elif pattern_id == "fos2":
            o = span_pos[0][0]
            f = span_pos[3][0]
        elif pattern_id == "fos3":
            o1 = span_pos[0][0] 
            o2 = span_pos[2][0]
            f = span_pos[3][0]
            return [(f,o1),(f,o2)]
        elif pattern_id == "fos4":
            o = span_pos[2][0]
            f = span_pos[0][0]
        elif pattern_id == "fos5":
            o = span_pos[2][0]
            f = span_pos[0][0]
        elif pattern_id == "fos6":
            o = span_pos[3][0]
            f = span_pos[0][0]
        elif pattern_id == "fos7":
            o = span_pos[0][0]
            f = span_pos[2][0]
        elif pattern_id == "fos8":
            o = span_pos[0][0]
            f = span_pos[2][0]
        elif pattern_id == "fos9":
            o = span_pos[1][0]
            f = span_pos[0][0]
        #-----------------------------------------------
        elif pattern_id == "p1":
            o = span_pos[0][0]
            f = span_pos[1][0]
        elif pattern_id == "p2":
            o = span_pos[0][0]
            f = "%s %s"%(span_pos[1][0] ,span_pos[2][0])
        elif pattern_id == "p3":
            o = "%s %s"%(span_pos[0][0] ,span_pos[1][0])
            f = span_pos[2][0]
        elif pattern_id == "p5":
            o = "%s %s"%(span_pos[1][0] ,span_pos[2][0])
            f = span_pos[3][0]
        elif pattern_id == "p6":
            o = "%s %s"%(span_pos[1][0] ,span_pos[2][0])
            f = span_pos[3][0]
        elif pattern_id == "p7":
            o = span_pos[1][0]
            f = span_pos[0][0]
#         elif pattern_id == "p8":
#             o = span_pos[1][0]
#             f = span_pos[0][0]
        elif pattern_id == "p9":
            o = span_pos[2][0]
            f = span_pos[0][0]
        else:
            return []
        
        if o in stopwords: return ()
        else: return [(f,o)]

    def run(self):
#         for sent in nltk.sent_tokenize(self.article):
#             sent = sent.lower()
#             sent = sent.replace(",", " ").replace(":", " ").replace("(", " ").replace(")", " ").replace("multi-",                                                                                                        "multiple ")
        self.article = self.article.replace(" is "," ")
        self.article = self.article.replace("  "," ")
        self.article = re.sub(r'(?:^| )\w(?:$| )', " ", self.article).strip() # remove single alphbet
        ress = self.match_pattern(self.article)
        return ress

In [13]:
# # sent_pattern = r"""<JJ>"""
# #     pattern = r'<PROPN>+ (<PUNCT|CCONJ> <PUNCT|CCONJ>? <PROPN>+)*'


# # tag_pattern = "<DT>?<JJ>*<NN.*>"
# # regexp_pattern = tag_pattern2re_pattern(tag_pattern)
# # regexp_pattern
# # '(<(DT)>)?(<(JJ)>)*(<(NN[^\\{\\}<>]*)>)'

# rev = """
# I found this model to be great value for money. It can play DVDs, VCDs, CDs, CD-RW (i.e. burned CDs) as well as MP3 files. It can also convert PAL to NTSC and NTSC to PAL on any TV, which enables you to watch foreign movies. Since it has 110-220 power coversion, you can use it anywhere in the US, Europe and Israel. It also has a secret menu that allows you to play movies from every DVD region number (look on the web, you'll find it...).The downside: seems like the production quality of these machines is problematic. The first unit I got did not work properly. I returned it with no problems, and the one I have now works great.Overall - a great buy, a fully load machine.
# """

# extract_pharse = []
# # Print all Verb Phrase
# extract_sents = sentence_extract_blob(rev)
# for rev_sent in extract_sents:
#     print(rev_sent)
#     print(get_pos_sequence(rev_sent))
#     doc = textacy.make_spacy_doc(rev_sent,lang='en_core_web_sm')
#     phrases = textacy.extract.matches(doc, [
#         [{"TAG": "NN"},{"TAG": "NN","OP": "+"}],
#         [{"TAG": "NN"},{"TAG": "VBZ","OP": "+"}]
#     ])
#     for phrase in phrases:
#     #         print("verb_phrases : " , phrase.text)
#     #     print([(token.text,token.pos_) for token in nlp(chunk.text)])
#         extract_pharse.append(phrase.text+"\n")           
# extract_pharse

# FOP-Rule(Dependency)

In [14]:
class FOP_rule_Depend():
    def __init__(self, review):
        self.review = review
        self.possible_verb = ['VB','MD','VBG','VBN']
        self.possible_adj = ["JJ","JJR"]
        self.possible_noun = ["NN","NNP","NNPS","NNS"]
        self.possible_adv = ['RB','RBR','RBS']

    def rule1(self, doc):
        #         amod(N, A) →< N, A >
        res = []
        for token_noun in doc:
            if token_noun.tag_ in self.possible_noun:
                for possible_opinion in token_noun.children:
                    if possible_opinion.dep == amod and possible_opinion.tag_ in self.possible_adj:
#                         print(token_noun.text,token_noun.tag_, possible_opinion.text,possible_opinion.tag_)
                        f,o = token_noun.text,possible_opinion.text
                        if o not in stopwords and f != o:
                            res.append((f,o))
                    #         print("rule1",res)
        return res

    def rule2(self, doc):
        #         acomp(V, A) + nsubj(V, N) →< N, A >
        res = []
        term1s, term2s = [], []
        for token_verb in doc:
            if token_verb.tag_ in self.possible_verb:
                for possible_opinion in token_verb.children:
                    if possible_opinion.dep == acomp and possible_opinion.tag_ in self.possible_adj:
#                         print(token_verb.text,token_verb.tag_, possible_opinion.text,possible_opinion.tag_)
                        term1s.append([token_verb.text, possible_opinion.text])  # acomp(V, A)
                    if possible_opinion.dep == nsubj and possible_opinion.tag_ in self.possible_noun:
                        term2s.append([token_verb.text, possible_opinion.text])  # nsubj(V, N)

        for t1 in term1s:
            V1, A1 = t1
            for t2 in term2s:
                V2, N2 = t2
                if V1 == V2: 
                    f,o = N2, A1
                    if o not in stopwords and f != o:
                        res.append((f,o))  # →< N, A >
            #         print("rule2",res)
        return res
    
    def rule3(self, doc):
        res = []
        #         nsubj(aux,noun) + acomp(aux,adj) →< N, A >

        term1s, term2s = [], []
        for token_AUX in doc:
            if token_AUX.pos == AUX:
                for possible_head in token_AUX.children:
                    if possible_head.dep == nsubj and possible_head.tag_ in self.possible_noun:
                        term1s.append([token_AUX.text, possible_head.text])  # nsubj(aux,noun)
                    if possible_head.dep == acomp and possible_head.tag_ in self.possible_adj:
                        term2s.append([token_AUX.text, possible_head.text])  # acomp(aux,adj)

        for t1 in term1s:
            Aux1, N1 = t1
            for t2 in term2s:
                Aux2, A2 = t2
                if Aux1 == Aux2 and N1 != Aux1 and Aux1 != "is": 
                    f,o = N1, Aux1
                    if o not in stopwords and f != o:
                        res.append((f,o))  # →< N, A >
            #         print("rule3",res)
        v = ""
#         if len(res) > 0: 
#             v = [get_pos_sequence(" ".join(r))[1] for r in res]
        return res


#     def rule4(self, doc):
#         res = []
#         #     dobj(V, N) + nsubj(V, N0) →< N, V >
#         term1s, term2s = [], []
#         for token_verb in doc:
#             if token_verb.tag_ in self.possible_verb:
#                 for possible_head in token_verb.children:
#                     if possible_head.dep == dobj and possible_head.tag_ in self.possible_noun:
# #                         print(token_verb.text,token_verb.tag_, possible_head.text,possible_head.tag_)
#                         term1s.append([token_verb.text, possible_head.text])  # nsubj(aux,noun)
#                     if possible_head.dep == nsubj and possible_head.tag_ in self.possible_noun:
#                         term2s.append([token_verb.text, possible_head.text])  # acomp(aux,adj)
#         for t1 in term1s:
#             V1, N1 = t1
#             f,o = V1, N1
#             if o not in stopwords and f != o:
#                 res.append((f,o))
#         # res.extend(term2s)
#         #         print("rule4",res)
#         return res

    def rule5(self, doc):
        #         < h1, m > +conj and(h1, h2) →< h2, m >
        res = []
        term1s, term2s = [], []
        for token in doc:
            for possible_head in token.children:
                if possible_head.tag_ in self.possible_noun + self.possible_adj + self.possible_adv:
                    term1s.append([token.text, possible_head.text])  # < h1, m >
                if possible_head.dep == conj and possible_head.tag_ in self.possible_noun:
                    term2s.append([token.text, possible_head.text])  # conj and(h1, h2)

        for t1 in term1s:
            h1, m = t1
            for t2 in term2s:
                h21, h22 = t2
                if h1 == h21:
#                     if m not in stopwords:
                    f,o = h1, m
                    if ((o not in stopwords) and (f not in stopwords)) and (f != o): 
                        res.append((f, o))
                    f,o = h22, m
                    if ((o not in stopwords) and (f not in stopwords)) and (f != o): 
                        res.append((f, o))                    
                #         print("rule5",res)
        return res

    def rule6(self, doc):
        #         < h, m1 > +conj and(m1, m2) →< h, m2 >
        #   nsubj(aux,noun) + acomp(aux,m1) + conj and(m1, m2) →< h, m1 > + < h, m2 >
        res = []
        term1s, term2s, term3s = [], [], []
        for token in doc:
            if token.pos == AUX:
                token_AUX = token
                for possible_head in token_AUX.children:
                    if possible_head.dep == nsubj and possible_head.tag_ in self.possible_noun:
                        term1s.append([token_AUX.text, possible_head.text])  # nsubj(aux,noun)
                    if possible_head.dep == acomp and possible_head.tag_ in self.possible_adj:
                        term2s.append([token_AUX.text, possible_head.text])  # acomp(aux,m1)
            else:
                for possible_m in token.children:
                    if possible_m.dep_ == "conj": term3s.append([token.text, possible_m.text])

        for t1 in term1s:
            Aux1, N1 = t1
            for t2 in term2s:
                Aux2, A2 = t2
                if Aux1 == Aux2 and N1 != Aux1:
                    #                     res.append([N1,A2]) # →< N, A >
                    for t3 in term3s:
                        m1, m2 = t3
                        if A2 in t3:
#                             if m1 not in stopwords and N1 != m1: 
                                f,o = N1,m1
                                if ((o not in stopwords) and (f not in stopwords)) and (f != o):     
                                    res.append((f, o))  # →< N, A > 
                                f,o = N1, m2
                                if ((o not in stopwords) and (f not in stopwords)) and (f != o):     
                                    res.append((f, o))  # →< N, A > 
                        #         print("rule6",res)
        return res

    def rule7(self, doc):
        #         < h, m > +neg(m, not) →< h, not + m >
        #          acomp(aux,adj) + neg(aux,part) + nsubj(aux,N1) + compound(N1,N2) => <N1 + N2 , part + adj>
        res = []
        term1s, term2s, term3s, term4s = [], [], [], []
        for token in doc:
            if token.pos == AUX:
                token_AUX = token
                for possible_head in token_AUX.children:
                    if possible_head.dep == acomp and possible_head.tag_ in self.possible_adj:
                        term1s.append([token_AUX.text, possible_head.text])  # acomp(aux,adj)
                    if possible_head.dep == neg and possible_head.pos == PART:
                        term2s.append([token_AUX.text, possible_head.text])  # neg(aux,part)
                    if possible_head.dep == nsubj and possible_head.tag_ in self.possible_noun:
                        term3s.append([token_AUX.text, possible_head.text])  # nsubj(aux,N1)
            else:
                for possible_m in token.children:
                    #                     print(token,possible_m,possible_m.dep_)
                    if possible_m.dep_ == "compound" and possible_m.tag_ in self.possible_noun:  # compound(N1,N2)
                        term4s.append([token.text, possible_m.text])

        for t1 in term1s:
            Aux1, adj = t1
            for t2 in term2s:
                Aux2, part = t2
                if Aux1 == Aux2:
                    #                     print(part , adj) # <part , adj>
                    for t3 in term3s:
                        Aux3, N1 = t3
                        if Aux1 == Aux3:
                            for t4 in term4s:
                                N41, N42 = t4
                                if N1 in t4:
#                                     res.append([N42, N41, part, adj])  # => <N1 + N2 , part + adj>
                                    f,o = "%s %s"%(N42, N41),"%s %s"%(part, adj)
                                    if (o not in stopwords) and (o not in f):
                                        res.append((f,o))  # => <N1 + N2 , part + adj>

                                #         print("rule7",res)
        return res

    def rule8(self, doc):
        #         < h, m > +nn(h, N) →< N + h, m >
        #       compound(h,N1) + nsubj(V1,N1) + acomp(V1,A) -> <h + N1 , A>
        res = []
        term1s, term2s, term3s = [], [], []
        for token in doc:
            for possible_head in token.children:
                if possible_head.dep_ == "compound" and possible_head.tag_ in self.possible_noun:
                    term1s.append([token.text, possible_head.text])  # compound(h,N1)
                if possible_head.dep == nsubj and token.tag_ in self.possible_verb and possible_head.tag_ in self.possible_noun:
                    term2s.append([token.text, possible_head.text])  # nsubj(V1,N1)
                if possible_head.dep == acomp and token.tag_ in self.possible_verb and possible_head.tag_ in self.possible_adj:
                    term3s.append([token.text, possible_head.text])  # acomp(V1,A)
        for t1 in term1s:
            N1, N2 = t1
            for t2 in term2s:
                V21, N22 = t2
                if N22 in t1:
                    #                     print(N2,N1) # <h + N1>
                    for t3 in term3s:
                        V31, A = t3
                        if V31 == V21:
                            f,o = "%s %s"%(N2, N1), A
                            if (o not in stopwords) and (o not in f):
                                res.append((f,o))  # -> <h + N1 , A>

                        #         print("rule8",res)
        return res

    def rule9(self, doc):
        #         < h, m > +nn(N, h) →< h + N, m >
        #         compound(N1,N2) + dobj(V,N1) -> < N2 + N1, V >
        res = []
        term1s, term2s = [], []
        for token in doc:
            for possible_head in token.children:
                if possible_head.dep_ == "compound" and possible_head.tag_ in self.possible_noun:
                    term1s.append([token.text, possible_head.text])  # compound(N1,N2)
                if possible_head.dep == dobj and token.tag_ in self.possible_adj + self.possible_verb and possible_head.tag_ in self.possible_noun:
#                     print(token.text,token.tag_, possible_head.text,possible_head.tag_)
                    term2s.append([token.text, possible_head.text])  # dobj(V,N1)
        for t1 in term1s:
            N1, N2 = t1
            for t2 in term2s:
                V, N22 = t2
                if N22 in t1:
                    f,o = "%s %s"%(N2, N1), V
                    if (o not in stopwords) and (o not in f):
                        res.append((f,o))  # -> <h + N1 , A>

                #         print("rule9",res)
        return res

    def run(self):
        self.review = self.review
        doc = nlp(self.review)
        #         displacy.render(doc, style='dep', jupyter = True) # dependency parse tree
        deplist = []
        dep1 = self.rule1(doc)
        dep2  = self.rule2(doc)
        dep3  = self.rule3(doc)
#         dep4  = self.rule4(doc)
        dep5  = self.rule5(doc)
        dep6  = self.rule6(doc)
        dep7  = self.rule7(doc)
        dep8  = self.rule8(doc)
        dep9  = self.rule9(doc)
#         print("dep1:",str(dep1))
#         print("dep2:",str(dep2))
#         print("dep3:",str(dep3))
#         print("dep4:",str(dep4))
#         print("dep5:",str(dep5))
#         print("dep6:",str(dep6))
#         print("dep7:",str(dep7))
#         print("dep8:",str(dep8))
#         print("dep9:",str(dep9))
#         print("-------------------------------------")
#         print(dep1 , dep2 , dep3 , dep4 , dep5 , dep6 , dep7 , dep8 , dep9)
        dep = dep1 + dep2 + dep3 + dep5 + dep6 + dep7 + dep8 + dep9
        return dep

In [15]:
class FOP_rule_Depend2():
    def __init__(self, review):
        self.review = review
        self.possible_verb = ['VB','MD','VBG','VBN']
        self.possible_adj = ["JJ","JJR"]
        self.possible_noun = ["NN","NNP","NNPS","NNS"]
        self.possible_adv = ['RB','RBR','RBS']
        # spacy 是head指向dependecy(child)
        # 一個word可以有很多child

    def rule10(self, doc):
        #  nsubj(f,o)
        res = []
        for token in doc:
            for children in token.children:
                if children.dep == nsubj:    
                    if children.tag in self.possible_noun:
    #                     print(token.text,token.tag_, children.text,children.tag_)
                        f,o = children,token
                        if (o.text in stopwords) or (f.text in stopwords): continue
                        if (o.text == f.text): continue
                        res.append((f.text,o.text))
                        #         print("rule10",res)
        return res

    
    def rule12(self, doc):
        #  amod(f,o)
        res = []
        for token in doc:
            for children in token.children:
                if children.dep == amod:                
#                     print(token.text,token.tag_, children.text,children.tag_)
                    f,o = token,children
                    if (o.text in stopwords) or (f.text in stopwords): continue
                    if (o.text == f.text): continue
                    res.append((f.text,o.text))
                    #         print("rule12",res)
        return res


    def rule13(self, doc):
        #  advmod(f,o)
        res = []
        for token in doc:
            for children in token.children:
                if children.dep == advmod and token.tag_ in self.possible_adj + self.possible_verb:                
#                     print(token.text,token.tag_, children.text,children.tag_)
                    f,o = token,children
                    if (o.text in stopwords) or (f.text in stopwords): continue
                    if (o.text == f.text): continue
                    res.append((f.text,o.text))
                    #         print("rule12",res)
        return res
    
    def rule14(self, doc):
        #  dobj(f,o)
        res = []
        for token in doc:
            for children in token.children:
                if children.dep == dobj and children.pos in [NOUN,PROPN] and token.tag_ in self.possible_verb:  
                    if (token.tag_ in self.possible_adj + self.possible_adv):#
                        f,o = children,token
                        if (o.text in stopwords) or (f.text in stopwords): continue
                        if (o.text == f.text): continue
                        res.append((f.text,o.text))
                        #         print("rule12",res)
        return res

    
    def run(self):
        self.review = self.review.replace(" is "," ")
        self.review = self.review.replace("  "," ")
        self.review = re.sub(r'(?:^| )\w(?:$| )', " ", self.review).strip() # remove single alphbet
#         print(self.review)
#         print(get_pos_sequence(self.review)[1])    
        doc = nlp(self.review)
#         displacy.render(doc, style='dep', jupyter = True) # dependency parse tree
        deplist = []
        dep10 = self.rule10(doc)
        dep12  = self.rule12(doc)
        dep13  = self.rule13(doc)
        dep14  = self.rule14(doc)        
#         print("dep1:",str(dep10))
#         print("dep3:",str(dep12))
#         print("dep4:",str(dep13))
#         print("dep5:",str(dep14))
        
#         print("-------------------------------------")
        dep = dep10 + dep12 + dep13 + dep14
        return dep
    
    
def TwoRuleFOPDep(rev_sent):
    fops1 = FOP_rule_Depend(rev_sent).run() 
    fops2 = FOP_rule_Depend2(rev_sent).run()
    fops = []
    [fops.append(fop) for fop in fops1 if fop not in fops]
    [fops.append(fop) for fop in fops2 if fop not in fops]    
    return fops

In [16]:
rev_sent = """
great for those with input receiver but no build decoder the amp
"""
DEP_fops = TwoRuleFOPDep(rev_sent)
DEP_fops

[]

In [17]:
rev = """
I was a happy S530D owner until a month ago when I began getting the dreaded \"C13 00\" error.  It would not recognize any DVD I put in; even brand new ones.\n\nI did a web search and was amazed to find that many, many people who have bought this player (and Sony players in general) have had exactly the same problem after a year to a year and half.\n\nSony refuses to do a recall and I've since bought a new player (not a Sony; I settled on a JVC XV-FA90), but I thought I should post my experience here.  Buyer beware on this player; it has a very high rate of terminal failure.
"""
# rev = """
# The screen is wide and clear.
# """

# rev = """
# I just purchased mine 11/5/99. I have had the evil lip-sync problem on only one movie(A bridge to far),Ive played about 8 DVDs. One other note, The universal remote isnt, when it comes to JVC receivers. It would have  been nice to control volume with the 530s remote. On the plus side it does  control my Sony TV perfectly.
# """
extract_sents = sentence_extract_blob(rev)
for rev_sent in extract_sents:
    POS_fops = FO_rule_POS(rev_sent).run()
    DEP_fops = TwoRuleFOPDep(rev_sent)
    if (len(POS_fops) == 0) and (len(DEP_fops) == 0): continue
    print(rev_sent)
#     print("POS:",POS_fops)
#     print("dep:",DEP_fops)
    print('---------------------------------------------')



was happy s d owner until month ago when began getting the dreaded c error.
---------------------------------------------
it would not recognize any dvd put in even brand new ones.
---------------------------------------------
did web search and was amazed to find that many many people who have bought this player and sony players in general have had exactly the same problem after year to year and half.
---------------------------------------------
sony refuses to do recall and i ve since bought new player not sony settled on jvc xv fa but thought should post my experience here.
---------------------------------------------
buyer beware on this player it has very high rate of terminal failure.
---------------------------------------------


# Rule Extract Sentence

In [18]:
rev = """
I found this model to be great value for money. It can play DVDs, VCDs, CDs, CD-RW (i.e. burned CDs) as well as MP3 files. It can also convert PAL to NTSC and NTSC to PAL on any TV, which enables you to watch foreign movies. Since it has 110-220 power coversion, you can use it anywhere in the US, Europe and Israel. It also has a secret menu that allows you to play movies from every DVD region number (look on the web, you'll find it...).The downside: seems like the production quality of these machines is problematic. The first unit I got did not work properly. I returned it with no problems, and the one I have now works great.Overall - a great buy, a fully load machine.
"""

# rev = SentProcess(rev)

# fops = FO_rule_POS(rev).run()
# print(fops,"\n")

# FOPS = FOP_rule_Depend(rev).run()
# print(FOPS)

# extract_sents = sentence_extract(rev)
# extract_sents


# print(keywords.keywords(rev))
# print(summarizer.summarize(rev))

# 提取所有規格關鍵字

In [19]:
import warnings
from tqdm import tqdm
from collections import Counter,OrderedDict

feature_counter1 = Counter()
feature_counter2 = Counter()
feature_counter3 = Counter()
warnings.filterwarnings('ignore')

import collections
freq_words = collections.Counter()
feat_words = collections.Counter()
opinion_words = collections.Counter()
cooccurs_words = collections.Counter()

_Sents = []
keywords_list = []
# 其中 freq_words 是單字出現的頻率 , _Sents 是文章中所有的句子

i = 0

# total_keywords = []
Products = len(PROD_DICT.items())
print("%s Products\n"%(Products))    

if not os.path.exists('View'):
    os.makedirs('View')
  

fn = 'View/%s_%s_keywords.txt'%(category1,category2)
if os.path.exists(fn):
    with open('View/Prod_keyword.txt',"w",encoding="utf-8") as f:
        # with open("Prod_keyword.txt","w",encoding="utf-8") as f:
        with tqdm(total=Products) as pbar:
            for asin,DATA_DICT in PROD_DICT.items():
                REVIEW_ITEM_LIST = DATA_DICT['REVIEW_ITEM_LIST']
                title = DATA_DICT['title']
                description = DATA_DICT['description']
                feature = DATA_DICT['feature']      
                dash_keywords,noun_keywords,newkeywords, newdescription = getKeywordFeat(description,feature)

                f.write("* asin:" + asin+"\n")
                if "(new Date()).getTime();" in title : f.write("title:" + asin+"\n")
                else: f.write("title:" + title+"\n")
                f.write('dash_keywords:\n' + str(dash_keywords)+"\n");f.write('noun_keywords2:\n' + str(noun_keywords)+"\n")
#                 keywords = noun_keywords
#                 keywords = list(list(dash_keywords1) + list(noun_keywords1))
#                 newkeywords1 = list()
#                 for key in list(dash_keywords1) + list(noun_keywords1):
#                     clean_key = remove_word2(key)
#                     clean_key = clean_key.split(" ")            
#                     newkeywords1.extend(clean_key) 
#                 newkeywords1 = list(set([key for key in newkeywords1 if len(key) >=2 and key not in stopwords and key not in opinion_lexicon["total-words"]]))
#                 keywords_list.append(" ".join(newkeywords1))
                f.write('* keywords:\n' + str(newkeywords)+"\n")
                f.write('* newdescription:\n' + str(newdescription)+"\n")
                f.write('* pos_newdescription:\n' + str(get_pos_sequence(newdescription)[1])+"\n")
                f.write('************************************************************************'+"\n")
                f.write('************************************************************************'+"\n")      
                for pair in REVIEW_ITEM_LIST:
                    review = pair["review"]           
                    vote = pair["vote"] 
                    summary = pair["summary"] 
#                     extract_sents = sentence_extract_blob(review)
#                     for rev_sent in extract_sents:            
#                         rev_sent,pos_sent = lemm_sent_process2(rev_sent, remove_stopwords=True, summary=False, mode="spacy", withdot=False)
#                         if pos_sent == None: continue
#                         if len(pos_sent) == 0 : continue 
#                         tok_sent = set(nltk.word_tokenize(rev_sent)) - set(stopwords)
#         #                     tok_sent = [token for token in nltk.word_tokenize(rev_sent) if token not in stopwords]
#                         freq_words.update(tok_sent)    
#                         _Sents.append(tok_sent)
                    dash_keywords2,noun_keywords2,newkeywords2, _ = getKeywordFeat_2(review)                    
                    dash_keywords3,noun_keywords3,newkeywords3, _ = getKeywordFeat_2(summary)
                feature_counter1.update(newkeywords)
                feature_counter2.update(newkeywords2)
                feature_counter3.update(newkeywords3)
                pbar.update(1)
                pbar.set_description("Processing %s" % asin)
                
    important_features = OrderedDict(sorted(feature_counter1.items(), key=lambda pair: pair[1], reverse=True))
    important_features = [(word,important_features[word]) for word in important_features if important_features[word]>5]
    print("Count : %s"%(len(important_features)))
    
    important_features2 = OrderedDict(sorted(feature_counter2.items(), key=lambda pair: pair[1], reverse=True))
    important_features2 = [(word,important_features2[word]) for word in important_features2 if important_features2[word]>5]
    print("Count : %s"%(len(important_features2)))
    
    important_features3 = OrderedDict(sorted(feature_counter3.items(), key=lambda pair: pair[1], reverse=True))
    important_features3 = [(word,important_features3[word]) for word in important_features3 if important_features3[word]>5]
    print("Count : %s"%(len(important_features3)))

    fn = 'View/%s_%s_keywords.txt'%(category1,category2)
    make = False
    if not os.path.exists(fn):
        make = True
        f = open(fn,"w",encoding="utf-8")

    total_keywords = set()
    # with open("total_keywords.txt",'w',encoding="utf-8") as f:
    for word , v in important_features:        
        if make: f.write("%s:%s \n"%(word,v))   
        if v > 20 : total_keywords.add(word)  
            
    fn2 = 'View/%s_%s_keywords2.txt'%(category1,category2)
    make = False
    if not os.path.exists(fn2):
        make = True
        f = open(fn2,"w",encoding="utf-8")

    total_keywords2 = set()
    # with open("total_keywords.txt",'w',encoding="utf-8") as f:
    for word , v in important_features2:        
        if make: f.write("%s:%s \n"%(word,v))   
        if v > 5 : total_keywords2.add(word)  
            
    fn3 = 'View/%s_%s_keywords3.txt'%(category1,category2)
    make = False
    if not os.path.exists(fn3):
        make = True
        f = open(fn3,"w",encoding="utf-8")

    total_keywords3 = set()
    # with open("total_keywords.txt",'w',encoding="utf-8") as f:
    for word , v in important_features3:        
        if make: f.write("%s:%s \n"%(word,v))   
        if v > 5 : total_keywords3.add(word)  
                
else:
    total_keywords = set()
    with open(fn,'r',encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            k,v = line.split(":")
            if int(v) > 20 :
                total_keywords.add(k)
            
# pbar.close()
    # total_keywords = set(total_keywords) # 檢查重要程度 篩選字典裡的單字
    # open("total_keywords.txt",'w',encoding="utf-8").write(str(total_keywords))

1618 Products



Processing B01HB7BZ26: 100%|██████████| 1618/1618 [1:40:56<00:00,  3.59s/it]  


Count : 963
Count : 494
Count : 33


# TF-IDF 關鍵字濾除 (each newkeywords)

In [20]:
# import os
# import sys
# from sklearn import feature_extraction
# from sklearn.feature_extraction.text import TfidfTransformer
# from sklearn.feature_extraction.text import CountVectorizer

# # def Importance(document, sorted_words):
# #     scores = {}
# #     sent = get_token_sent(document)
# #     for word in sent:
# #         try:
# #             scores[word] = dict(sorted_words)[word]
# #         except Exception as e:
# #             pass
# #     scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
# #     return scores[:20]

# vectorizer = CountVectorizer()  # 该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频
# transformer = TfidfTransformer()  # 该类会统计每个词语的tf-idf权值
# tfidf = transformer.fit_transform(
#     vectorizer.fit_transform(keywords_list))  # 第一个fit_transform是计算tf-idf，第二个fit_transform是将文本转为词频矩阵
# word = vectorizer.get_feature_names()  # 获取词袋模型中的所有词语
# weight = tfidf.toarray()  # 将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重

# print("Calculate TF-IDF finished")

# scores = {}
# for i in range(len(weight)):
#     # print (u"-------这里输出第",i,u"类文本的词语tf-idf权重------")
#     for j in range(len(word)):
#         # print(word[j],weight[i][j])
#         scores[word[j]] = weight[i][j]


# # fout = 'TF-IDF/%s_%s.txt'%(big_categories,small_categories)
# if not os.path.isdir("TF-IDF"):
#     os.mkdir("TF-IDF")
    
# with open('TF-IDF/%s_%s.txt'%(category1,category2),'w',encoding='utf-8') as f:   
#     f.write("word" + '\t' + "tf-idf-weight" + '\n')
#     for k, v in scores.items():
#         if float(v) <= 0: continue
# #         print(k,v)
#         f.write(str(k) + '\t' + str(v) + '\n')

# # sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
# sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
# sorted_words = [word[0] for word in sorted_words]
# print("\nTop 20 Important words", sorted_words[:20])

# # total_keywords = set(sorted_words)

# 自然語言處理 -- Pointwise Mutual Information

In [21]:
# 使用total_keywords計算pmi
from math import log
# need pmi feature words & opinions
def p(x): # p(x) 計算單字 x 出現的機率
    return freq_words[x]/float(len(freq_words))

def pxy(x,y): # pxy(x,y) 計算單字 x 和單字 y 出現在同一個句子的機率
#     print((lambda s :  x in s and y in s ,_Sents))
#     filter()函數用於過濾序列，過濾掉不符合條件的元素
    return (len(list(filter(lambda s :  x in s and y in s ,_Sents)))+1)/ float(len(_Sents) )

def pmi(x,y): # pmi(x,y) 計算單字 x 和單字 y 的 Pointwise Mutual Information
    try:
#         print("X :%s Y: %s"%(x,y))
#         print('pxy(x,y)',pxy(x,y))
#         print('p(x)',p(x))
#         print('p(y)',p(y))
#         print('log(pxy(x,y)/(p(x)*p(y)),2)',log(pxy(x,y)/(p(x)*p(y)),2))
        return  log(pxy(x,y)/(p(x)*p(y)),2) 
    except Exception as e :
#         print(e)
        return 0
    
def pmi_fopair(sentence):
    doc = nlp(sentence)
    noun_,op_ = [],[]
    cand_pairs = []
    cand_pairs_score = []
    for token in doc:
        if token.pos in [ADJ,ADV,VERB]: op_.append(token.text)
        if token.pos in [NOUN]: noun_.append(token.text)

    for f in noun_:
        for o in op_:
            pair = (f,o)
            score = pmi(f,o)
#             if score > 0: cand_pairs.append((pair,score))
            if score > 0: cand_pairs.append(pair)
            cand_pairs_score.append((pair,score))

#             cand_pairs.append((pair,score))
#     print(sentence)
#     print(cand_pairs)
    return cand_pairs,cand_pairs_score

In [22]:
# _Sents[3]

# p('year')
# p('close')

# pmi_fopair("player close year absolutly amazing piece equipment")

# 計算FO-PAIR以及句子抽取

In [23]:
import pandas as pd
i = 0    

if not os.path.exists('Train'):
    os.makedirs('Train')
    
pattern_text = open("View/review_pattern.txt",'w',encoding="utf-8")
# training_data = open("View/%s_%s_train.txt"%(category1,category2),'w',encoding="utf-8")
# training_data.write("rev_sent | polarity,subjectivity | f1_o1 f2_o2 f3_o3 | summary" + '\n') 

topic_train_df = {}
key_train_df = {}
i = 0
j = 0
with tqdm(total=Products) as pbar:
    with open("View/result.txt",'w',encoding="utf-8") as result:
        for asin,DATA_DICT in PROD_DICT.items():
            REVIEW_ITEM_LIST = DATA_DICT['REVIEW_ITEM_LIST']
            title = DATA_DICT['title']
#             if "(new Date()).getTime();" in title : continue
            description = DATA_DICT['description']
            feature = DATA_DICT['feature']      
            if "(new Date()).getTime();" not in title :result.write("title:" + title+"\n")
            else: result.write("title:" + asin+"\n")

            result.write('************************************************************************'+"\n")      
            for pair in REVIEW_ITEM_LIST:
                review = pair["review"]  
                rev_dash_keywords, newreview = cleanReview(review)
#                 f.write('newreview:\n'+newreview+"\n")
                vote = pair["vote"]  
                overall = pair["overall"] 
                summary = pair["summary"]
                review_ID = pair["review_ID"]
                summ_token_set = set(nltk.word_tokenize(summary))
#                 if len(summ_token_set & total_keywords) == 0: continue
                result.write('overall:' + str(overall) + "\n")
                result.write('review:\n' + review + "\n")
                result.write('summary:\n' + summary + "\n")
                result.write("************************************************\n")
     
                extract_sents = sentence_extract_blob(review)
                keyword_list = []
                rev_summ_row = {}
                for rev_sent in extract_sents:
                    fop_row = {} # fop train row
                    fop_row['overall'] = overall
                    sentiment = TextBlob(rev_sent).sentences[0].sentiment
                    
#                     rev_sent = rev_sent.replace("s.","")
#                     rev_sent = re.sub(r'(?:^| )\w(?:$| )', " ", rev_sent).strip() # remove single alphbet
#                     rev_sent = rev_sent.replace(" ve "," ").replace(" ha "," ")                  
                    lemm_sent,pos_sent = lemm_sent_process2(rev_sent, remove_stopwords=False, summary=False, mode="spacy", withdot=False)#                     _,pos_sent = get_pos_sequence(rev_sent,remove_stopwords=False,summary=False,mode = "spacy",withdot =True)
#                     if len(pos_sent) == 0: continue
                    token_set = set(nltk.word_tokenize(rev_sent))
                    mention_features = list(token_set & total_keywords)                   
                    POS_fops = FO_rule_POS(lemm_sent).run()
                    DEP_fops = FOP_rule_Depend(lemm_sent).run()
                    POS_fops = [(f,o) for f,o in POS_fops if f in total_keywords]
                    DEP_fops = [(f,o) for f,o in DEP_fops if f in total_keywords]
#                     PMI_fops,cand_pairs_score = pmi_fopair(lemm_sent) # 皆為空有問題                     

                    result.write("************************************************\n")                     
                    result.write('rev_sent:\n' + rev_sent + "\n") ;fop_row['rev_sent'] = rev_sent
                    result.write('lemm_sent:\n' + lemm_sent) ;fop_row['lemm_sent'] = lemm_sent
                    
#                     result.write(str(pos_sent)+"\n");pattern_text.write(str(pos_sent)+"\n") ; 
                    result.write('mention features:' + str(mention_features) + "\n")
                    result.write("polarity: %s subjectivity: %s \n"%(sentiment.polarity,sentiment.subjectivity))
                    fop_row['polarity'] = sentiment.polarity;fop_row['subjectivity'] = sentiment.subjectivity
                    fop_row['mention_features'] = ",".join(mention_features)
                
                    result.write("POS_fops:"+str(POS_fops)+"\n") ;pattern_text.write("POS_fops:"+str(POS_fops)+"\n")
                    result.write("DEP_fops:"+str(DEP_fops)+"\n") ;pattern_text.write("DEP_fops:"+str(DEP_fops)+"\n")
#                     f.write("PMI_fops:"+str(cand_pairs_score)+"\n")
#                     print(cand_pairs_score)
                    result.write("************************************************\n") 
                    if "not" in token_set: continue              
                    if overall > 3 and sentiment.polarity < 0: continue
                    if overall < 3 and sentiment.polarity > 0: continue
                    if (len(POS_fops) == 0) and (len(DEP_fops) == 0): continue
                        
#                     fop_list = set(PMI_fops + POS_fops + DEP_fops)
                    fop_list = set(POS_fops + DEP_fops)
                    fop_prob_list = []
                    
                    for fop in fop_list:
                        p1,p2,p3 = 0,0,0; y1,y2,y3 = 0.6,0.2,0.2;
                        feat,opinion = fop
#                         if fop in PMI_fops: 
#                             p1 = pmi(feat,opinion)
                        if fop in POS_fops: p2 = 1
                        if fop in DEP_fops: p3 = 1
#                         p = y1*p1 + y2*p2 + y3*p3
                        p = y2*p2 + y3*p3
                        fop_prob_list.append((fop,p))
                    result.write("fop_prob_list:" + str(fop_prob_list)+ "\n")   
                    i +=1 
                    result.write("\n");pattern_text.write("\n")
                    # ------------------------------------------------------
#                     training_data.write("%s"%(rev_sent) + "|") 
#                     training_data.write("%s|%s|"%(sentiment.polarity,sentiment.subjectivity))
#                     if len(token_set & total_keywords) == 0: continue                    
                    
                    fop_str = ""
                    for idx,fop_score in enumerate(fop_prob_list):
                        fop,score = fop_score
                        feat,o = fop                        
                        if idx == len(fop_prob_list) - 1:
                            fop_str += "%s %s"%(feat,o)
                        else:     
                            fop_str += "%s %s\t|\t"%(feat,o)
                    fop_row["fops"] = fop_str 
                    keyword_list.append(fop_str)
                    fop_row['review_ID'] = review_ID
                    fop_row['summary'] = summary
                    
                    if abs(sentiment.polarity) <= 0.5: continue
                    if abs(sentiment.subjectivity) < 0.4: continue
                    if len(mention_features) == 0: continue
                    topic_train_df[i] = fop_row ;i +=1       
                
                    
            total_keyword = " ".join(keyword_list)
            rev_summ_row['review_ID'] = review_ID
            rev_summ_row['review'] = review
            rev_summ_row['summary'] = summary
            rev_summ_row['overall'] = overall
            rev_summ_row['total_keyword'] = total_keyword
            key_train_df[j] = rev_summ_row ; j += 1                       
#                 f.write("---------------------------------------------------\n")
            pbar.update(1)
            pbar.set_description("Processing %s" % asin)
#             break  
            
print("finished...")
# training_data.close()
topic_train_df = pd.DataFrame.from_dict(topic_train_df, orient='index')
key_train_df = pd.DataFrame.from_dict(key_train_df, orient='index')

Processing B01HB7BZ26: 100%|██████████| 1618/1618 [2:48:36<00:00,  6.32s/it]  


finished...


In [24]:
csv_path = "Train/%s_%s_topic.xlsx"%(category1,category2)
# df.to_csv(csv_path) #默认dt是DataFrame的一个实例，参数解释如下
topic_train_df.to_excel(csv_path, encoding='utf8')
print(csv_path + " Write finished")
print(" Write finished")
topic_train_df.head()

Train/Electronics_DVD Players_topic.xlsx Write finished
 Write finished


,overall,rev_sent,lemm_sent,polarity,subjectivity,mention_features,fops,review_ID,summary
5,5.0,tried popping in the matrix and the fifth elem...,try pop the matrix and the element and reward ...,0.90,0.90,matrix,matrix element\t|\tmatrix pop,957225600,Feature-packed with a Brillian Picture
7,5.0,the sd produces just as brilliant sound to com...,the produce just as brilliant sound to complim...,0.65,0.70,"video,sound",sound brilliant\t|\tvideo compliment,957225600,Feature-packed with a Brillian Picture
9,5.0,very nice feature is the hdcd decoder which al...,very nice feature the hdcd decoder which allow...,0.74,0.95,"feature,decoder",sound superior\t|\tfeature very nice\t|\tfeatu...,957225600,Feature-packed with a Brillian Picture
16,5.0,the progressive scan makes an incredible diffe...,the progressive scan make incredible difference\n,0.90,0.90,"scan,difference",difference incredible,957225600,Feature-packed with a Brillian Picture
20,5.0,all can say is that was blown away the picture...,all can say that blow away the picture quality...,0.90,0.90,"scan,quality,dvd,picture",dvd progressive,956966400,Stunning Picture Quality - Easy To Use


In [25]:
csv_path = "Train/%s_%s_key.xlsx"%(category1,category2)
# df.to_csv(csv_path) #默认dt是DataFrame的一个实例，参数解释如下
key_train_df.to_excel(csv_path, encoding='utf8')
print(csv_path + " Write finished")
print(" Write finished")

Train/Electronics_DVD Players_key.xlsx Write finished
 Write finished


In [26]:
key_train_df.head()

,review_ID,review,summary,overall,total_keyword
0,956966400,I bought the Toshiba SD-5109 as a Christmas pr...,Stunning Picture Quality - Easy To Use,5.0,toshiba buy functionality progressive\t|\toutp...
1,959385600,"This is an awesome DVD player, and I haven't e...",Ultimate DVD Player,5.0,channel tv\t|\tvolume channel\t|\tvolume tv\t|...
2,954201600,Purchased this unit in September 99. It worked...,Great Value,5.0,unit purchase machine lock lock parental\t|\tm...
3,940982400,The Sony DVPS530D DVD Player rocks my world! J...,SonyDVPS530D Rocks My World! Buy Now!!!!!,5.0,
4,964483200,I recieved the Palmtheatre free through a cont...,Anywhere you want,5.0,product nice\t|\tproduct convenient
